# Topology changes  

These methods are intended to provide the user with the ability to modify a `ToyTree` object pragmatically and efficiently without the need to directly modify `TreeNode` objects. While manual editing of TreeNodes is possible, it is not reccommended due to the danger of creating a `ToyTree` with invalid coordinate structure. These functions ensure that the `ToyTree` objects remain intact after topology changes such that when used in combination, trees can be safely modified to fit the structure required for any vizualization or algorithmic analysis while remaining compatible with the rest of the `Toytree` package's methods.  

The foundation of topology modification consists of adding, removing, or changing the relationships among TreeNodes. There also exists methods to work on the subtree level to make separation particularly efficient, as well as methods to generally restructure trees for informative visual clarity.

These methods are organized into three categories:  

1. `Node-level modification` 
2. `Subtree-level modification`
3. `Tree-level modification`

## Node-level modification

### Adding nodes  

The `mod` subpackage includes many methods to add nodes to a `ToyTree` object. A `Node`can be added as an internal node, child node, or sister node. You can also add nodes as a parent-child pair or as entire subtrees to be merged into a `ToyTree` object. 


`add_internal_node` introduces a new `Node` object to the tree by splitting the edge above a queried Node. It creates a new unary Node along this edge, with a name passed in as an argument `name=`.


In [129]:
import toytree
#simple ToyTree object from newick string
tree = toytree.tree("(B,(A,X)AX)AB;")
tree.draw(
    node_mask = False,
    node_sizes = 16,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
);
#add internal node named "C" ABOVE node "A"
tree2 = tree.mod.add_internal_node("A", name="C", dist=None)
tree2.draw(
    node_mask = False,
    node_sizes = 16,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t3e5abd5609a140bb9d168d2479657864"> B A X AX AB B A X

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="td369ef7f6683457886e8437ee12e971b"> B X A C AX AB B X A

Here, the `dist=` argument determines how much of the original child-parent distance between the queried `Node` and its parent will belong to the new child-parent distance created with `add_internal_node()` (parent being the newly introduced `Node`). A double $0<x<1$ is passed in which represents the proportion of the original distance being transfered to the new distance. See below how the graph above changes as we manipulate the `dist=` argument.

In [130]:
#assign 75% of the original distance to the new tip-parent distance
tree3 = tree.mod.add_internal_node("A", name="C", dist=0.75)
#assign 25% of the original distance to the new tip-parent distance
tree4 = tree.mod.add_internal_node("A", name="C", dist=0.25)

tree3.draw(
    node_mask = False,
    node_sizes = 16,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
);
tree4.draw(
    node_mask = False,
    node_sizes = 16,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tef7f38bab5ff4c32a92dfddb53246cdd"> B X A C AX AB B X A

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t7afda97757c9428cadffaeb2bd0a9ca6"> B X A C AX AB B X A


`add_child_node`works similarly to, and is a great function to pair with `add_internal_node`. This inserts a `Node` as a `child` to a queried Node within a tree such that, for example, a newly inserted internal node would no longer be unary. This can also create a polyploidy if the function is called on a binary `Node`.


In [131]:
import toytree
#simple ToyTree object from newick string
tree = toytree.tree("(B,(A,X)AX)AB;")
tree.draw(
    node_mask = False,
    node_sizes = 16,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
);
#add internal node named "C" ABOVE node "A"
tree2 = tree.mod.add_internal_node("A", name="C", dist=None)
tree2.draw(
    node_mask = False,
    node_sizes = 16,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
);
#add child node to new internal node "C" to make binary
tree3 = tree2.mod.add_child_node("C", name="D", dist=None)
tree3.draw(
    node_mask = False,
    node_sizes = 16,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="td0772e3f1f5a4ba2a6dabe9fcadf5ef6"> B A X AX AB B A X

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tf27fc66ef4bb43ad841bb1078e6386ac"> B X A C AX AB B X A

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t82cf2c2f4522468ca447b57361c22db0"> B X A D C AX AB B X A D

In [132]:
#add another child node to "C", creating a polyploidy
polyploid = tree3.mod.add_child_node("C", name="Y")

canvas, axes, mark = polyploid.draw(
    node_mask = False,
    node_sizes = 16,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
);


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t4dd4627257a54d2ca25c0f011264aa65"> B X A D Y C AX AB B X A D Y


`add_sister_node` can also be used as a method to use in conjunction to `add_internal_node`, the only difference being which node is queried. This function inserts a `Node` as the sister of a queried Node. In other words, it adds a child Node to the parent of the queried Node. Similarly, this can also either fix a unary node to become binary, or case a binary node to become a polyploidy.


In [133]:
import toytree
#simple ToyTree object from newick string
tree = toytree.tree("(B,(A,X)AX)AB;")
tree.draw(
    node_mask = False,
    node_sizes = 16,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
);
#add internal node named "C" ABOVE node "A"
tree2 = tree.mod.add_internal_node("A", name="C", dist=None)
tree2.draw(
    node_mask = False,
    node_sizes = 16,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
);
#add sister node to node "A" to make "C" binary
tree3 = tree2.mod.add_child_node("C", name="D", dist=None)
tree3.draw(
    node_mask = False,
    node_sizes = 16,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t1fafa1c2db5b4af3862913044f7f4316"> B A X AX AB B A X

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t40d027b268e24174bbdac64e744022bc"> B X A C AX AB B X A

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t6630e9c7480047f8b155b59162029d8c"> B X A D C AX AB B X A D

In [134]:
#add another sister node to "A", creating a polyploidy
polyploid = tree3.mod.add_sister_node("A", name="Y")

polyploid.draw(
    node_mask = False,
    node_sizes = 16,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t19aa95d6d8c843b7a5da8486dd4adc25"> B X A D Y C AX AB B X A D Y


`add_internal_node_and_child` is a method to combine these previous steps into one command. When used, a parent-child pair is passed into the tree, splitting the edge above the queried Node. The internal Node and new child Node must both be defined with `parent_name=` and `name=` respectively. If no value is entered for `parent_dist`, then the parent Node is inserted at the midpoint of the edge. If a `parent_dist` value is defined, then it must fit within the length of the query Node's dist. The new child Node does not share these constraints. If no value is entered for `dist`, then it will be set to match the dist of its sister Node.


In [135]:
import toytree
#simple ToyTree object from newick string
tree = toytree.tree("(B,(A,X)AX)AB;")
tree.draw(
    node_mask = False,
    node_sizes = 16,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
);
#add internal node named "C" with child node named "D" ABOVE node "A"
tree2 = tree.mod.add_internal_node_and_child("A", name="D", parent_name="C")
tree2.draw(
    node_mask = False,
    node_sizes = 16,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t719cb8778f4d4627ad44f1413adf4614"> B A X AX AB B A X

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t9907073dde9b430e9879ae7a50b458b0"> B X A D C AX AB B X A D


`add_internal_node_and_subtree` takes the previous function one step further by allowing the user to pass in a subtree. Similar to the other functions, this splits the edge between the queried node and its parent, but this time splits it with a new ancestral node to which a subtree (passed in as a `ToyTree` object) is connected. The name of the ancestral Node is passed in to the `parent_name=` argument, and both the distance of the parent and the subtree stem can be set with `parent_dist=` and `subtree_stem_dist=` respectively. By default, these are set at $0.5$. You can also choose to rescale the subtree such that it fits in the distance between the sister Node height and stem height.

In [136]:
import toytree

tree = toytree.tree("(A,(B,C));")
sub = toytree.tree("(X,(Y,Z));")
#add subtree "sub" to original tree above node C
merged = tree.mod.add_internal_node_and_subtree("C", 
                                                subtree=sub,
                                                parent_name="D", 
                                                subtree_rescale=True)
tree.draw();
sub.draw();
merged.draw();


<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t800ce47f0fb142669f7670d7fb7e7a77"> A B C

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tb5240cce8230408f94d84e3e5783cc8f"> X Y Z

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tddc6c3c87cb946b3918d3a3e7793893e"> A B C X Y Z

### Removing nodes  

`remove_nodes` simply deletes the nodes that are queried. By default, the orphans created by deleting internal nodes (perhaps this metaphor has gone too far) inheret their deleted parents' distances such that their distances reaches their grandparent's original height. The user can alternatively pass in `preserve_dists=False` to have children retain their original distances (while still being connected to their grandparents). 

In [137]:
import toytree 

tree = toytree.tree("(a,b,((c,d)CD,(e,f)EF)X)AB;")
mod_tree = tree.mod.remove_nodes("b", "c", "EF")
tree.draw();
mod_tree.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t583c482bd8024eaeb904514413825041"> a b c d e f

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tbc190b11ec8c46328c29f660be81c33b"> a d e f

### Removing unary nodes  

The _`.mod`_ subpackage also offers `remove_unary_nodes()`, a method to quickly remove all `unary Nodes`, or Nodes that have exactly 1 child. This way, the ToyTree object returned will ony contain tips and internal Nodes with $\geq 2$ children. This method does not take in any arguments other than the ToyTree object (unless called using `tree.mod.remove_unary_nodes()`) and `inplace=` to determine whether or not to modify the original tree or make a copy.

In [150]:
import toytree

tree = toytree.tree("(A,(B,C)X)Y;")
tree = tree.mod.add_child_node("C", name="E").mod.add_child_node("B", name="F").mod.add_child_node("A", name="G")
tree.draw(
    node_mask = False,
    node_sizes = 16,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
    );
simplified = tree.mod.remove_unary_nodes().mod.rotate_node("Y")
simplified.draw(
    node_mask = False,
    node_sizes = 16,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="td4185284769d49d6ab67826df0292637"> G F E A B C X Y G F E

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t20ad11e3ac144c16b238297587eacd6a"> G F E X Y G F E

### Collapsing nodes  

`mod.collapse_nodes()` can be called on an internal node to collapse it into a multi-furcating polytomy. This can either be done by passing in particular Node labels, or by providing a minimum distance `min_dist` or minimum support value `min_support`. These represent the minimum value allowed for the Node to _stay_. That is - every internal Node with value _less than_ the min value provided will be collapsed.

In [204]:
import toytree

tree = toytree.tree("(A,(B,(C,D)X)Y)Z;")
#modifying tree with previous methods, adding complexities
tree = tree.mod.add_child_node("C", name="E").mod.add_child_node("B", name="F").mod.add_internal_node_and_child("A", name="G", parent_name="H")
tree.draw(
    node_mask = False,
    node_sizes = 11,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
    );
#collapsing specific nodes by name - collapse X and H
collapsed1 = tree.mod.collapse_nodes("X", "H")
collapsed1.draw(
    node_mask = False,
    node_sizes = 11,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t0948d73703ba47ffbc43d73ddb443917"> F E D A G B C X Y H Z F E D A G

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t82e84c67751a4a96816cd7bc3ef91740"> F E D A G B C Y Z F E D A G

In [206]:
import toytree

tree = toytree.tree("(A,((B,E)H,((C,G)J,(D,F)K)X)Y)Z;")
#setting distances to be different values
tree.set_node_data("dist", {7: 3, 8: 4, 9: 5}, inplace=True)
tree.draw(
    tree_style = 'c', #coalescent style to see distance values
    node_mask = False,
    node_sizes = 10,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
    );
#keep only internal nodes with parental edge length >1.5
#this will only get rid of a few internal nodes with particularly branch lengths
collapsed3 = tree.mod.collapse_nodes(min_dist=1.5)
collapsed3.draw(
    tree_style = 'c',
    node_mask = False,
    node_sizes = 10,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
);
#keep only internal nodes with parental edge length > 5, collapse the rest.
#in this case, K is the only internal node with edge length >5
collapsed4 = tree.mod.collapse_nodes(min_dist = 5)
collapsed4.draw(
    tree_style = 'c',
    node_mask = False,
    node_sizes = 10,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="name",
    node_labels_style= {"fill": "white"}
);

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t9654f67ca07647479d563b6f1c7cab03"> A B E C G D F H J K X Y Z A B E C G D F 0 2.5 5 7.5

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tf1fbb0cf66bf40709f80362128a77513"> A B E C G D F H J K Z A B E C G D F 0 2.5 5 7.5

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t4ce55f70656044e386b3ca616eaab17f"> A B E D F C G K Z A B E D F C G 0 2.5 5 7.5

### Rotating nodes  

`mod.rotate_node()` rotates a particular node such that the order of its children are reversed. A node can be queried using either the index or the name, and internal nodes can be accessed by passing in multiple nodes, which will rotate the node representing their `MRCA`.  

By default, this returns a modified copy of the tree passed in without chainging the originial tree, however `inplace=True` will change and return the original tree passed in.  


In [ ]:
import toytree 
#simple tree from newick string
tree = toytree.tree("(Alligator,(Bunny,(Cat,Dog)X)Y)Z;")
tree.draw(
    node_mask = False,
    node_sizes = 16,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="idx",
    node_labels_style= {"fill": "white"}
    );
#rotate tree at node idx=4 (first internal node)
tree2 = toytree.mod.rotate_node(tree, 4)
tree2.draw(
    node_mask = False,
    node_sizes = 16,
    node_markers = "s",
    node_style= {"fill": "black"},
    node_labels="idx",
    node_labels_style= {"fill": "white"}
);
#look for the difference in order between Cat and Dog

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t575c6bcec7a94c88b46d415cf5a8184f"> 0 1 2 3 4 5 6 Alligator Bunny Cat Dog

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t8e5b5ae5923c40df98e4816c66615e3f"> 0 1 2 3 4 5 6 Alligator Bunny Dog Cat

Multiple calls of `rotate_node` can be chained to efficiently change specific formatting.

In [ ]:
#more complex newick string
tree = toytree.tree("(a,((b,c)BC,(d,(e,f))DE)X)AB;") 
tree.draw();
#multiple calls chained together, accessing internal nodes by MRCA of two tips
rotated = tree.mod.rotate_node('c', 'd').mod.rotate_node('f','d')
rotated.draw();

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t998e62d96b704596a72ef3bce4236de2"> a b c d e f

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="275.0px" viewBox="0 0 300.0 275.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t6167af239f464ffda142b52b7b983667"> a e f d b c

## Subtree-level manipulation

### Pruning  

Updates coming soon.

### Bisecting  

Updates coming soon.

## Tree-level modification

### Resolving polytomies  

Updates coming soon.

### Ladderize  

Updates coming soon.